# Classification of MIS Status

In [302]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import seaborn as sns


# (1) Python code that:
    cleans the training and test data files
    converting the data into two separate Pandas data frames of numerical data



In [272]:
# load train, test, column descriptors
df_train = pd.read_excel("SBA_training_data.xlsx")
df_test = pd.read_excel("SBA_test_data.xlsx")
df_columns = pd.read_csv("column_descriptions.csv")

In [273]:
df_train.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997-02-28,1997,...,N,Y,NaT,1999-02-28,60000,0,P I F,0,60000,48000
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997-02-28,1997,...,N,Y,NaT,1997-05-31,40000,0,P I F,0,40000,32000
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997-02-28,1997,...,N,N,NaT,1997-12-31,287000,0,P I F,0,287000,215250
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997-02-28,1997,...,N,Y,NaT,1997-06-30,35000,0,P I F,0,35000,28000
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997-02-28,1997,...,N,N,NaT,1997-05-14,229000,0,P I F,0,229000,229000


In [274]:
df_test.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1069795008,"Maple Dye, Inc.",LOS ANGELES,CA,90005,BANK OF AMERICA NATL ASSOC,NC,424690,2004-09-27,2004,...,T,N,NaN,2004-09-30,66869,0,NaN,NaN,50000,25000
1,1069835007,"Resource Solutions, Corp.",COLUMBUS,WI,53925,BMO HARRIS BK NATL ASSOC,IL,541519,2004-09-27,2004,...,N,N,NaN,2005-03-31,100000,0,NaN,NaN,100000,85000
2,1069855002,Shadow Horse LLC,BRADLEY BEACH,NJ,7720,BANK OF AMERICA NATL ASSOC,RI,236118,2004-09-27,2004,...,N,N,NaN,2004-10-31,18000,0,NaN,NaN,18000,9000
3,1069865005,Cold Mountain Company dba Robe,HOUSTON,TX,77098,WOODFOREST NATIONAL BANK,TX,453920,2004-09-27,2004,...,Y,N,NaN,2004-10-31,387018,0,NaN,NaN,200000,100000
4,1069875008,"Double K Equipment Sales, Inc.",Minden,LA,71055,CAPITAL ONE NATL ASSOC,VA,423820,2004-09-27,2004,...,N,N,NaN,2004-10-31,25000,0,NaN,NaN,25000,12500


## Clean Training Data

In [275]:
# The following variables are categorical and not numerical:
df_train["Zip"] = df_train["Zip"].astype('str')
df_train["FranchiseCode"] = df_train["FranchiseCode"].astype('str')
df_train["Name"] = df_train["Name"].astype('str')
df_train["Bank"] = df_train["Bank"].astype('str')
df_train["BankState"] = df_train["BankState"].astype('str')
df_train["RevLineCr"] = df_train["RevLineCr"].astype('str')
df_train["MIS_Status"] = df_train["MIS_Status"].astype('str')

# The following variables were dropped because: 
df_train_x = df_train.drop(labels=['ChgOffPrinGr', # data leakage
                                   'MIS_Status', # data leakage
                                   'ChgOffDate', # data leakage
                                   "ApprovalDate", # won't generalize to future data
                                   "DisbursementDate", # won't generalize to future data
                                   "LoanNr_ChkDgt"], # unique randomly generated IDs
                           axis=1)
df_train_y = df_train["MIS_Status"]
#df_train[np.isnan(df_train["MIS_Status"])]

## Clean Test Data

In [276]:
# The following variables are categorical and not numerical:
df_test["Zip"] = df_test["Zip"].astype('str')
df_test["FranchiseCode"] = df_test["FranchiseCode"].astype('str')
df_test["Name"] = df_test["Name"].astype('str')
df_test["Bank"] = df_test["Bank"].astype('str')
df_test["BankState"] = df_test["BankState"].astype('str')
df_test["RevLineCr"] = df_test["RevLineCr"].astype('str')
df_test["MIS_Status"] = df_test["MIS_Status"].astype('str')

# The following variables were dropped because: 
df_test_x = df_test.drop(labels=['ChgOffPrinGr', # data leakage
                                   'MIS_Status', # data leakage
                                   'ChgOffDate', # data leakage
                                   "ApprovalDate", # won't generalize to future data
                                   "DisbursementDate", # won't generalize to future data
                                   "LoanNr_ChkDgt"], # unique randomly generated IDs
                           axis=1)
df_test_y = df_test["MIS_Status"]

## Encode training data into numerical variables

In [277]:
d = defaultdict(LabelEncoder)
# Encoding the variable
fit = df_train.apply(lambda x: d[x.name].fit_transform(x))
# Inverse the encoded
fit.apply(lambda x: d[x.name].inverse_transform(x))

le_y = LabelEncoder()
le_y.fit(df_train["MIS_Status"].astype(str))
df_train_encoded_y = le_y.transform(df_train["MIS_Status"].astype(str))


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:285: FutureWarning: In the future, NAT != NAT will be True rather than False.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not em

In [278]:
# Use label encoder dictionary to encode training data
df_train_encoded_x = df_train_x.apply(lambda x: d[x.name].fit_transform(x))
df_train_encoded_x.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,140,1022,15,2532,345,35,484,5,84,4,...,0,0,1,0,2,2,1358,0,280,428
1,5006,2297,15,2489,19,15,766,5,60,2,...,0,0,1,0,2,2,943,0,193,295
2,9424,272,15,2525,471,15,711,5,161,7,...,0,0,1,0,2,1,2684,0,744,956
3,949,338,36,3771,3,36,0,5,60,2,...,0,0,1,0,2,2,827,0,163,262
4,328,2469,9,1559,442,9,0,5,194,14,...,7,7,1,0,2,1,2560,0,665,979


## Sanity check: See if inverse encoding is the same as original data

In [279]:
df_train_encoded_x.head().apply(lambda x: d[x.name].inverse_transform(x))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997,84,4,...,0,0,1,0,N,Y,60000,0,60000,48000
1,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997,60,2,...,0,0,1,0,N,Y,40000,0,40000,32000
2,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997,180,7,...,0,0,1,0,N,N,287000,0,287000,215250
3,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997,60,2,...,0,0,1,0,N,Y,35000,0,35000,28000
4,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997,240,14,...,7,7,1,0,N,N,229000,0,229000,229000


In [280]:
df_train_x.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,1997,84,4,...,0,0,1,0,N,Y,60000,0,60000,48000
1,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,1997,60,2,...,0,0,1,0,N,Y,40000,0,40000,32000
2,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,1997,180,7,...,0,0,1,0,N,N,287000,0,287000,215250
3,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,1997,60,2,...,0,0,1,0,N,Y,35000,0,35000,28000
4,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,1997,240,14,...,7,7,1,0,N,N,229000,0,229000,229000


## Encode test data using label encoders of test data

In [281]:
df_test_encoded_x = df_test_x.apply(lambda x: d[x.name].fit_transform(x))
df_test_encoded_x.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,619,343,3,760,15,24,134,2,17,10,...,0,0,1,1,2,0,215,0,76,73
1,772,124,46,423,25,13,254,2,73,8,...,0,0,1,2,1,0,259,0,114,144
2,833,63,30,606,15,36,14,2,35,1,...,0,0,1,1,1,0,57,0,22,20
3,237,267,41,604,157,40,188,2,73,4,...,0,0,1,1,3,0,385,0,158,155
4,311,412,17,538,32,42,116,2,73,3,...,0,0,1,1,1,0,90,0,39,37


In [282]:
df_test_encoded_x.head().apply(lambda x: d[x.name].inverse_transform(x))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,"Maple Dye, Inc.",LOS ANGELES,CA,90005,BANK OF AMERICA NATL ASSOC,NC,424690,2004,18,10,...,0,0,1,1,T,N,66869,0,50000,25000
1,"Resource Solutions, Corp.",COLUMBUS,WI,53925,BMO HARRIS BK NATL ASSOC,IL,541519,2004,84,8,...,0,0,1,2,N,N,100000,0,100000,85000
2,Shadow Horse LLC,BRADLEY BEACH,NJ,7720,BANK OF AMERICA NATL ASSOC,RI,236118,2004,36,1,...,0,0,1,1,N,N,18000,0,18000,9000
3,Cold Mountain Company dba Robe,HOUSTON,TX,77098,WOODFOREST NATIONAL BANK,TX,453920,2004,84,4,...,0,0,1,1,Y,N,387018,0,200000,100000
4,"Double K Equipment Sales, Inc.",Minden,LA,71055,CAPITAL ONE NATL ASSOC,VA,423820,2004,84,3,...,0,0,1,1,N,N,25000,0,25000,12500


In [283]:
df_test_x.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalFY,Term,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,"Maple Dye, Inc.",LOS ANGELES,CA,90005,BANK OF AMERICA NATL ASSOC,NC,424690,2004,18,10,...,0,0,1,1,T,N,66869,0,50000,25000
1,"Resource Solutions, Corp.",COLUMBUS,WI,53925,BMO HARRIS BK NATL ASSOC,IL,541519,2004,84,8,...,0,0,1,2,N,N,100000,0,100000,85000
2,Shadow Horse LLC,BRADLEY BEACH,NJ,7720,BANK OF AMERICA NATL ASSOC,RI,236118,2004,36,1,...,0,0,1,1,N,N,18000,0,18000,9000
3,Cold Mountain Company dba Robe,HOUSTON,TX,77098,WOODFOREST NATIONAL BANK,TX,453920,2004,84,4,...,0,0,1,1,Y,N,387018,0,200000,100000
4,"Double K Equipment Sales, Inc.",Minden,LA,71055,CAPITAL ONE NATL ASSOC,VA,423820,2004,84,3,...,0,0,1,1,N,N,25000,0,25000,12500


# (2) "MIS_Status" is to be predicted
    ChgOffDate = The date when a loan is declared to be in default
    MIS_Status = Loan status charged off = CHGOFF, Paid in full = PIF
    ChgOffPrinGr = charged-off amount

In [284]:
#df_train_x = df_train.drop(labels=['ChgOffPrinGr','MIS_Status','ChgOffDate',"ApprovalDate",
#                                   "DisbursementDate","LoanNr_ChkDgt"],axis=1)
#df_train_y = df_train["MIS_Status"]

In [285]:
#df_test["Zip"] = df_test["Zip"].astype('str')
#df_test["FranchiseCode"] = df_test["FranchiseCode"].astype('str')
#df_test["ApprovalFY"].value_counts()
#df_test_x = df_test.drop(labels=['ChgOffPrinGr','MIS_Status','ChgOffDate',"ApprovalDate",
#                                   "DisbursementDate","LoanNr_ChkDgt"],axis=1)
#df_test_y = df_test["MIS_Status"]
#df_test
#df_test_encoded_x = df_test_x.astype(str).apply(LabelEncoder().fit_transform)

In [286]:
#from sklearn.preprocessing import LabelEncoder
#le_city = LabelEncoder()
#le_city.fit(df_train["City"])
#le_city.classes_
    
#le_zip = LabelEncoder()
#le_zip.fit(df_train["Zip"])
#le_zip.transform(df_train["Zip"])

#le_afy = LabelEncoder()
#le_afy.fit(df_train["ApprovalFY"])
#le_afy.transform(df_train["ApprovalFY"])

#le_y = LabelEncoder()
#le_y.fit(df_train["MIS_Status"].astype(str))
#le_y.transform
#le_y.classes_

#df_train.apply(LabelEncoder().fit_transform)
#df_train_encoded_x = df_train_x.astype(str).apply(LabelEncoder().fit_transform)

In [287]:
#df_train_encoded_x.head()

In [288]:
#x_tmp = le_afy.transform(df_train["ApprovalFY"])
#y_tmp = le_y.transform(df_train["MIS_Status"].astype(str))

#pd.Series(y_tmp).value_counts()

(3)  At least three potential functions / models for predicting (even if badly) the chosen data variable(s),

In [289]:
# train decision tree

clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf.fit(X=df_train_encoded_x, y=df_train_encoded_y)

In [290]:
# train random forest

clf_rf = RandomForestClassifier(n_estimators=500, random_state=0)
clf_rf.fit(X=df_train_encoded_x, y=df_train_encoded_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [291]:
# train logistic regression
clf_log = LogisticRegression(random_state=0,
                            solver='lbfgs',
                            multi_class='multinomial')
clf_log.fit(df_train_encoded_x,df_train_encoded_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

# (4)  Predicted values of the chosen data variable(s) over both the training and test sets

In [300]:
# training set predictions
train_pred_tree = clf_tree.predict(df_train_encoded_x)
train_pred_log = clf_log.predict(df_train_encoded_x)
train_pred_rf = clf_rf.predict(df_train_encoded_x)

In [301]:
# test set predictions
test_pred_tree = clf_tree.predict(df_test_encoded_x)
test_pred_log = clf_log.predict(df_test_encoded_x)
test_pred_rf = clf_rf.predict(df_test_encoded_x)

# (5)  5-fold cross-validated accuracy was used to assess training set accuracy 

In [303]:
# evaluate decision tree
cv_tree = cross_val_score(clf_tree,df_train_encoded_x, df_train_encoded_y, cv=5)

In [304]:
# evaluate logistic regression
#clf_log.score(df_train_encoded_x,df_train_encoded_y)
cv_log = cross_val_score(clf_log,df_train_encoded_x, df_train_encoded_y, cv=5)

In [305]:
# evaluate random forest
cv_rf = cross_val_score(clf_rf,df_train_encoded_x, df_train_encoded_y, cv=5)

(6)  Documentation of all of these findings as well as any conclusions obtained,

In [319]:
df_cv = pd.DataFrame([cv_tree,cv_log,cv_rf])
df_cv.index = ['Decision Tree', 'Logistic Regression', 'Random Forest']
df_cv.to_excel("cross_validated_accuracy.xlsx")

(7)  At least eight (8) PowerPoint slides (to be submitted on Sunday, 28 October 2018) to accompany ~10 minute in-class oral presentations of results (to be presented on Monday, 29 October 2018).

As discussed in class, Item (1) may include such things as creating "one-hot vectors" for categorical variables and / or appending additional quantitative information from outside sources (which must be cited).  The potential functions / models discussed in Item (3) can be as simple as a function that one generates from one's own intuition / by guessing or as complex as a machine learning model (e.g., a Perceptron, a Decision Tree, a Random Forest, etc.).

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,